In [1]:
#*******************************************************************************************
 #
 #  File Name:  student_loans.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, student_loans.ipynb, reads a csv file,
 #      student_loans.csv, and uses deep learning methods to process the features 
 #      in the provided dataset and create a binary classifier that can predict 
 #      whether student loan applicants will default or not.
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  04/15/2024      Initial Development                     Nicholas J. George
 #
 #******************************************************************************************/

import logx
import pandas_processx
import student_loans_constants

import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

2024-04-17 10:00:05.458448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
CONSTANT_LOCAL_FILE_NAME = 'student_loans.ipynb'


logx.set_log_mode(False)

logx.set_image_mode(False)


logx.begin_program('student_loans')

# <br> **Section 1: Extraction**

## **1.1: Read the CSV data into a Pandas DataFrame**

In [3]:
student_loan_dataframe = pd.read_csv(student_loans_constants.CONSTANT_INPUT_FILE_PATH)

logx.log_write_object(student_loan_dataframe)

## **1.2: Display Student Loan DataFrame**

In [4]:
pandas_processx.return_formatted_table(student_loan_dataframe, 'Table 1.2: Student Loan Table')

payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5
7.80,0.88,0.00,2.60,0.10,25.00,67.00,1.00,3.20,0.68,9.80,5
7.80,0.76,0.04,2.30,0.09,15.00,54.00,1.00,3.26,0.65,9.80,5
11.20,0.28,0.56,1.90,0.07,17.00,60.00,1.00,3.16,0.58,9.80,6
7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40,5
7.40,0.66,0.00,1.80,0.07,13.00,40.00,1.00,3.51,0.56,9.40,5
7.90,0.60,0.06,1.60,0.07,15.00,59.00,1.00,3.30,0.46,9.40,5
7.30,0.65,0.00,1.20,0.07,15.00,21.00,0.99,3.39,0.47,10.00,7
7.80,0.58,0.02,2.00,0.07,9.00,18.00,1.00,3.36,0.57,9.50,7
7.50,0.50,0.36,6.10,0.07,17.00,102.00,1.00,3.35,0.80,10.50,5


# <br> **Section 2: Preprocessing**

## **2.1: Create the labels series (`y`)  from the “spam” column, and then create the features (`X`) DataFrame from the remaining columns.**

### **Separate the Y Variable, the Labels**

In [5]:
y_series = student_loan_dataframe['credit_ranking']

logx.log_write_object(y_series)

In [6]:
y_series.value_counts()

credit_ranking
5    681
6    638
7    199
4     53
8     18
3     10
Name: count, dtype: int64

### **Separate the X Variable, the Features**

In [7]:
x_dataframe = student_loan_dataframe.drop(columns = 'credit_ranking', axis = 1)

logx.log_write_object(x_dataframe)

In [8]:
pandas_processx.return_formatted_table(x_dataframe, 'Table 2.1: Student Loan Features DataFrame')

payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40
7.80,0.88,0.00,2.60,0.10,25.00,67.00,1.00,3.20,0.68,9.80
7.80,0.76,0.04,2.30,0.09,15.00,54.00,1.00,3.26,0.65,9.80
11.20,0.28,0.56,1.90,0.07,17.00,60.00,1.00,3.16,0.58,9.80
7.40,0.70,0.00,1.90,0.08,11.00,34.00,1.00,3.51,0.56,9.40
7.40,0.66,0.00,1.80,0.07,13.00,40.00,1.00,3.51,0.56,9.40
7.90,0.60,0.06,1.60,0.07,15.00,59.00,1.00,3.30,0.46,9.40
7.30,0.65,0.00,1.20,0.07,15.00,21.00,0.99,3.39,0.47,10.00
7.80,0.58,0.02,2.00,0.07,9.00,18.00,1.00,3.36,0.57,9.50
7.50,0.50,0.36,6.10,0.07,17.00,102.00,1.00,3.35,0.80,10.50


## **2.2: Split the Data into Training and Testing Datasets by Using `train_test_split`.**

In [9]:
x_train_dataframe, x_test_dataframe, \
y_train_series, y_test_series \
    = train_test_split \
        (x_dataframe, y_series, 
         random_state = student_loans_constants.CONSTANT_DL_RANDOM_STATE_1)

In [10]:
logx.log_write_object(x_train_dataframe)

logx.log_write_object(x_test_dataframe)

logx.log_write_object(y_train_series)

logx.log_write_object(y_test_series)

## **2.3: Use the StandardScaler to Scale the X Variables**

### **Create a StandardScaler Instance**

In [11]:
current_standard_scalar = StandardScaler()

### **Fit the StandardScaler**

In [12]:
x_standard_scalar = current_standard_scalar.fit(x_train_dataframe)

### **Scale the Data**

In [13]:
x_train_scaled_nparray = x_standard_scalar.transform(x_train_dataframe)

logx.log_write_object(x_train_scaled_nparray)

In [14]:
x_test_scaled_nparray = x_standard_scalar.transform(x_test_dataframe)

logx.log_write_object(x_test_scaled_nparray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [15]:
number_input_features_integer = len(x_train_scaled_nparray[0])

logx.print_and_log_text \
    ('\033[1m'
     + 'The number of inputs (features) in the model is {:,}.' \
         .format(number_input_features_integer) 
     + '\033[0m')

The number of inputs (features) in the model is 11.


In [16]:
input_layer_units_integer = 98

input_layer_activation_string = 'tanh'

logx.print_and_log_text \
    ('\033[1m'
     + 'The number of nodes in the input layer is {:,}.' \
         .format(input_layer_units_integer) 
     + '\033[0m')

The number of nodes in the input layer is 98.


In [17]:
hidden_layer_units_integer = 45

hidden_layer_activation_string = 'tanh'

logx.print_and_log_text \
    ('\033[1m'
     + 'The number of nodes in the hidden layer is {:,}.' \
         .format(hidden_layer_units_integer) 
     + '\033[0m')

The number of nodes in the hidden layer is 45.


In [18]:
output_layer_units_integer = 1

output_layer_activation_string = 'linear'

logx.print_and_log_text \
    ('\033[1m'
     + 'The number of nodes in the output layer is {:,}.' \
         .format(output_layer_units_integer) 
     + '\033[0m')

The number of nodes in the output layer is 1.


### **Instantiate the Model**

In [19]:
neural_network_sequential_model = tf.keras.models.Sequential()

### **Layers**

In [20]:
neural_network_sequential_model.add \
    (tf.keras.layers.Dense \
         (units = input_layer_units_integer, 
          activation = input_layer_activation_string, 
          input_dim = number_input_features_integer))

neural_network_sequential_model.add \
    (tf.keras.layers.Dense \
         (units = hidden_layer_units_integer, 
          activation = hidden_layer_activation_string))

neural_network_sequential_model.add \
    (tf.keras.layers.Dense \
         (units = output_layer_units_integer,
          activation = output_layer_activation_string))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### **Model Summary**

In [21]:
neural_network_sequential_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 98)             │         1,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 45)             │         4,455 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            46 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,677 (22.18 KB)

 Trainable params: 5,677 (22.18 KB)

 Non-trainable params: 0 (0.00 B)

### **Compile**

In [22]:
neural_network_sequential_model.compile \
    (loss = 'mean_squared_error', 
     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0024876),
     metrics = ['mse'])

## **3.2: Fit and Train Model**

In [23]:
earlystopping_callback \
    = tf.keras.callbacks.EarlyStopping  \
        (monitor = 'val_mse', mode = 'min', patience = 100, restore_best_weights = True)

neural_network_sequential_model \
    .fit \
        (x_train_scaled_nparray,
         y_train_series.values,
         epochs = 1000,
         validation_data = (x_test_scaled_nparray, y_test_series.values),
         callbacks = [earlystopping_callback])

Epoch 1/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 27.2998 - mse: 27.2998 - val_loss: 3.9058 - val_mse: 3.9058
Epoch 2/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.6391 - mse: 1.6391 - val_loss: 0.5227 - val_mse: 0.5227
Epoch 3/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5387 - mse: 0.5387 - val_loss: 0.4219 - val_mse: 0.4219
Epoch 4/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5087 - mse: 0.5087 - val_loss: 0.4277 - val_mse: 0.4277
Epoch 5/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4693 - mse: 0.4693 - val_loss: 0.4246 - val_mse: 0.4246
Epoch 6/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4396 - mse: 0.4396 - val_loss: 0.4055 - val_mse: 0.4055
Epoch 7/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4518 - mse: 0.4518 - val_loss: 0.4082 - val_mse: 0.4082
Epoch 8/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4027 - mse: 0.4027 - val_loss: 0.3923 - val_mse: 0.3923
Epoch 9/1000
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

## **3.3: Evaluate Model**

In [24]:
model_loss_float, model_mse_float \
    = neural_network_sequential_model.evaluate(x_test_scaled_nparray, y_test_series.values, verbose = 2)

logx.print_and_log_text \
    (f'\nModel Loss: {round(model_loss_float * 100, 2)}%, '
     + f'Model MSE: {round(model_mse_float * 100, 2)}%')

13/13 - 0s - 1ms/step - loss: 0.3396 - mse: 0.3396

Model Loss: 33.96%, Model MSE: 33.96%


## **3.4: Save and Export Model**

In [25]:
neural_network_sequential_model.save(student_loans_constants.CONSTANT_NN_MODEL_FILE_PATH)

# <br> **Section 4: Predict Loan Repayment Success**

## **4.1: Reload Model**

In [26]:
reloaded_neural_network_sequential_model \
    = tf.keras.models.load_model \
        (student_loans_constants.CONSTANT_NN_MODEL_FILE_PATH)

## **4.2: Predictions**

In [27]:
predictions_nparray \
    = reloaded_neural_network_sequential_model.predict \
        (x_test_scaled_nparray).round().astype('int32')

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


## **4.3: Compare Predictions and Actual Values**

In [28]:
comparison_dataframe \
    = pd.DataFrame({'predictions': predictions_nparray.ravel(), 'actual': y_test_series.values})

In [29]:
pandas_processx.return_formatted_table \
    (comparison_dataframe, 'Table 4.3: Model Predictions vs. Actual Values')

predictions,actual
6,6
6,6
5,5
6,6
6,6
6,7
5,4
6,5
5,5
5,5


In [30]:
# logx.end_program()